# Analyzing Reg-Seq from first principles
## 2023-07-26

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt

import bokeh.io
import bokeh.plotting
import holoviews as hv

import iqplot

bokeh.io.output_notebook()
hv.extension('bokeh')

# import panel as pn
# pn.extension()

# data preprocessing
from sklearn.model_selection import train_test_split

# linear regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## The Dataset

We have processing sequencing data (barcodes) from recent Reg-Seq runs. For example, Tom's "promoter long" experiment:

In [ ]:
dataset = Path('~/projects/regseq_analysis/data/tom_apr23/p260_barcodes.csv')
df = pd.read_csv(dataset)
df

Before we dive in, let's get a sense for what our dataset is like.
- `ct_0` is the number of times the promoter DNA was sequenced. `ct_1` is the number of times the corresponding RNA read was sequenced. `ct` is the sum (less useful).
- The `promoter` sequence is a 160 bp stretch (we should check this) from which we eventually hope to predict expression.
- In many cases we know which gene the promoter has been scavenged from; this is shown in the `name`.

There's more metadata here, but I don't think it's all going to be immediately useful (and I'd need to talk to Tom anyways to make sure I know what it all means).

Forging ahead with validation, let's first check that all the promoter sequences are indeed equal in length; this will be necessary for later model development -- for instance, any neural networks will expect inputs of equal length.

In [ ]:
# check that all sequences have the same length
df['promoter'].apply(len).unique()

Great! Not only is every promoter equal in length, but that length is also the (anticipated) 160 bp.

Let's get a sense of the distribution of expression across all rows of the dataset. First, we look at `ct_0` and `ct_1` independently:

In [ ]:
hist_ct0 = iqplot.histogram(data=df, q='ct_0')
hist_ct1 = iqplot.histogram(data=df, q='ct_1')

bokeh.io.show(bokeh.layouts.gridplot([hist_ct0, hist_ct1], ncols=1))

Hm, this isn't ideal -- the dataset is pretty big, so these histograms are quite unwieldy (especially for the RNA counts). There also looks to be a strange, regular pattern along the x-axis in both, which I think isn't noise but rather an effect of overplotting. Will have to find a better way to do this. At least the distributions seem skewed right, which is what we expect: most sequences don't show up in either sequencing run.

Next, we need a value that will represent **gene expression**. A simple first idea is to take the ratio `ct_1/ct_0` for each row. We should, I think, discard any rows for which `ct_0` was zero, since this will break down there.

In [ ]:
# count how many rows have ct_0 = 0
print("There are", df['ct_0'].value_counts().iloc[0], "rows with ct_0 = 0")
print("Deleting them...")

# delete rows for which ct_0 is 0
df = df[df['ct_0'] != 0]

# add a column 'expression' that is the ratio of ct_1/ct_0
df['expression'] = (df['ct_1']/df['ct_0'])

df

In [ ]:
bokeh.io.show(iqplot.ecdf(data=df, q='expression'))


## Test case: araBp

It ought to be more informative to compare results within a single gene (that's how we'll have to train our models anyway). Which genes have the most rows of data associated with them?

In [ ]:
# count how many unique genes there are
gene_names = df.name.unique().tolist()
print(len(gene_names), "unique genes identified.")

# sort by number of rows
grouped = df.groupby(['name']).size().reset_index()
grouped.sort_values([0], ascending=False)

As we can see, after filtering out sequences for which no DNA reads were found, the most prevalent promoters are araBp, xylAp, rspAp, znuCp, and xylFp. (I'm not sure what `*` means here.)

What does the distribution look like _within_ araBp?

In [ ]:
df_araBp = df.loc[df['name'] == 'araBp']
df_araBp

In [ ]:
bokeh.io.show(iqplot.histogram(data=df_araBp, q='expression'))

It's still difficult to see anything useful with all this data overplotted in a single place. Will have to return to this.

### One-hot encoding

Before we can start training anything on sequence data, we should _one-hot encode_ our sequences to convert them into numerical, rather than categorical, data. This will ensure any model doesn't learn to assign undue significance to the relative ordering of bases that doesn't have any basis in biological reality.

In [ ]:
# create an integer mapping of DNA bases
base_to_index = {"A": 0, "T": 1, "C": 2, "G": 3}

def one_hot_encode(seq):
    '''One-hot encode a given DNA sequence.'''
    # preallocate matrix
    encoded_seq = np.zeros((len(seq), len(base_to_index)), dtype=int)
    
    # loop over seq and encode each base
    for i, base in enumerate(seq):
        encoded_seq[i, base_to_index[base]] = 1
        
    return encoded_seq


# for every sequence in the dataframe, one-hot encode it
df_araBp['one_hot'] = df_araBp['promoter'].apply(one_hot_encode)
df_araBp

## Linear regression

To begin our lowbrow attempt at Reg-Seq analysis, let's start with the simplest possible case: linear regression.

Our one-hot encoded DNA sequences will be `x`, and the resulting gene expression will be `y`.

First, let's split our data into training and testing sets:

In [ ]:
# split data for training vs. testing
x = np.stack(df_araBp['one_hot'].values)
y = df_araBp['expression'].values

# reshape x into a 2D array as needed
n_samples, len_seq, n_bases = x.shape
x = np.reshape(x, (n_samples, len_seq * n_bases))

print(x.shape, y.shape)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# create & train a linear regression model
model = LinearRegression()
model.fit(x_train, y_train)

# make predictions on the testing data
y_pred = model.predict(x_test)

# evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# retrieve the learned weights and intercept
weights = model.coef_
intercept = model.intercept_


Here, the weights tell us the relative impact of each nucleotide position on gene expression. Let's see them:

In [ ]:
print(weights)